Detección de caras con webcam con detectores de OpenCV

In [ ]:
import cv2
import numpy as np

# Función para superponer una imagen con canal alfa (transparente) sobre el frame
def overlay_image_alpha(img, overlay, pos_x, pos_y, overlay_size):
    overlay = cv2.resize(overlay, overlay_size)  # Redimensiona la imagen de overlay

    # Validar si la imagen de overlay tiene un canal alfa
    if overlay.shape[2] == 4:
        # Dividir en canales BGR y Alfa
        b, g, r, a = cv2.split(overlay)
        overlay_rgb = cv2.merge((b, g, r))
        mask = cv2.merge((a, a, a))
    else:
        # Si no tiene canal alfa, usamos una máscara totalmente opaca
        overlay_rgb = overlay
        mask = np.ones_like(overlay_rgb) * 255

    h, w = overlay_rgb.shape[:2]
    roi = img[pos_y:pos_y+h, pos_x:pos_x+w]  # Región de interés

    # Aplicar la máscara para superponer
    img[pos_y:pos_y+h, pos_x:pos_x+w] = np.where(mask == 255, overlay_rgb, roi)

# Detector de Opencv2
haar_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Carga las imágenes
peluca_img = cv2.imread('peluca_payaso.png', cv2.IMREAD_UNCHANGED)  
nariz_img = cv2.imread('nariz_payaso.png', cv2.IMREAD_UNCHANGED)  

# Capturando con la webcam
video_capture = cv2.VideoCapture(0)

while True:
    ret, frame = video_capture.read()    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = haar_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        peluca_size = (w, h)
        nariz_size = (int(w * 0.5), int(h * 0.5))
        peluca_x, peluca_y = x, y - peluca_size[1] // 2
        nariz_x, nariz_y = x + w // 2 - nariz_size[0] // 2, y + h // 2 - nariz_size[1] // 2

        # Superponer la peluca y la nariz
        overlay_image_alpha(frame, peluca_img, peluca_x, peluca_y, peluca_size)
        overlay_image_alpha(frame, nariz_img, nariz_x, nariz_y, nariz_size)
    
    cv2.imshow('Filtro Payaso', frame)

    key = cv2.waitKey(1)
    if key == 27:  # Presiona Esc para salir
        break

# Cierra la ventana y la camara
video_capture.release()
cv2.destroyAllWindows()